In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.models import load_model
import pm4py
import pandas as pd
from PIL import Image
import numpy as np
import glob
from abc import ABC, abstractmethod

In [2]:
# получаем уникальные активити
activities = ['Start', 'Ist', 'A', 'B', 'Icmp', 'T', 'End', 'C', 'E', 'D']
act_dict = {}

# проводим  между номером и активити
act_num = len(activities)
for i in range(act_num):
  act_dict[activities[i]] = i

In [3]:
class Converter(ABC):
  
  def __init__(self, log_file_path, ch_sep, caseID_col_name, act_col_name, ts_col_name):
        # Динамические поля (переменные объекта)
        self.caseID_col_name = caseID_col_name 
        self.act_col_name = act_col_name
        self.ts_col_name = ts_col_name

        #self.event_log = pd.read_csv(log_file_path, sep=ch_sep)
        self.event_log = pm4py.read_xes(log_file_path)
        self.event_log[caseID_col_name]=self.event_log[caseID_col_name].apply(str)
        self.event_log[act_col_name]=self.event_log[act_col_name].apply(str)
        #self.event_log[ts_col_name]= pd.to_datetime(self.event_log[ts_col_name])

        self.event_log = self.event_log[[act_col_name, caseID_col_name, ts_col_name]]

  @abstractmethod
  def convert(self):
    pass


In [4]:
class ActivityConverter(Converter):

  def __get_unique_ids__(self):
    ids = list(self.event_log[self.caseID_col_name].unique())
    return ids

  def __get_case_logs__(self, ids_list):
    case_logs = []
    for id in ids_list:
      case_log = self.event_log.query("`{0}` == @id".format(self.caseID_col_name))
      case_log.sort_values(by=[self.ts_col_name])
      case_logs.append(case_log)
    return case_logs

  def __get_prefix_traces__(self):
    cases_prefix_traces = []
    self.max_len_prefix_trace = 0

    for case_log in self.case_logs:
      prefix_traces_act = []
      for i in range(1, len(case_log)+1):
        prifix_trace = case_log[self.act_col_name].values[0:i]

        #находим самую длинную префиксную трассу
        if (len(prifix_trace) > self.max_len_prefix_trace):
          self.max_len_prefix_trace = len(prifix_trace)

        prefix_traces_act.append(prifix_trace)

      cases_prefix_traces.append(prefix_traces_act)
    return cases_prefix_traces

  def __get_activity_matrices__(self, act_num, act_dict):
    activity_matrices = []
    for prefix_traces in self.cases_prefix_traces:
      np_matrix = []
      matrix = [ [0]*act_num for i in range(self.max_len_prefix_trace)]
      for i in range(len(prefix_traces)):
        for act in prefix_traces[i]:
          act_index = act_dict[act]
          matrix[i][act_index] += 1
          np_matrix = np.asmatrix(matrix)
          np_matrix = np_matrix.astype("uint8")
      activity_matrices.append(np_matrix)
    return activity_matrices


  def convert(self, path_to_dir):
    ids_list = self.__get_unique_ids__()
    self.case_logs = self.__get_case_logs__(ids_list)
    self.cases_prefix_traces = self.__get_prefix_traces__()
  
    self.activity_matrices = self.__get_activity_matrices__(act_num, act_dict)
  
    index = 1
    for np_matrix in self.activity_matrices:
      norm_matrix = np_matrix.copy()
      norm_matrix *= 255.0/norm_matrix.max()
      A = np.squeeze(np.asarray(norm_matrix)) 
      img = Image.fromarray(A)

      img = img.resize((224, 224), Image.NEAREST)
      path = path_to_dir + "/image_" + str(index) + ".png"
      index+=1
      img.save(path)      
     

In [7]:
act_conv = ActivityConverter('data_4/test_net_4_RD_EAU_patterns.xes', ',', "concept:instance", "concept:name", "time:timestamp")

act_conv.convert('data_4/Test_net_4_images')

parsing log, completed traces :: 100%|████████████████████████████████████████████| 1000/1000 [00:03<00:00, 309.54it/s]
D:\CoureseProject\ResponseNeverLogClassification\courseproject\lib\site-packages\pm4py\objects\log\util\dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
D:\CoureseProject\ResponseNeverLogClassification\courseproject\lib\site-packages\pm4py\objects\log\util\dataframe_utils.py:176: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], utc=True)
D:\CoureseProject\ResponseNeverLogClassification\courseproject\lib\site-packages\pm4py\objects\log\util\dataframe_utils.py:176: UserWarning: Could not infer format,

In [13]:
class_names = tf.keras.utils.image_dataset_from_directory('data_4/patterns').class_names
class_names

Found 4000 files belonging to 4 classes.


['pattern_dependent_exst_sub',
 'pattern_direct',
 'pattern_existence_act_universal',
 'pattern_never']

In [10]:
model = load_model(os.path.join('models','pattern_model_4.h5'))

In [ ]:
classes = []
files = glob.glob("data_4/Test_net_4_images/*.png")
for myFile in files:
    image = cv2.imread(myFile)
    resize = tf.image.resize(image, (256,256))
    yhat = model.predict(np.expand_dims(resize/255, 0))
    classes.append(class_names[np.argmax(yhat)])
# for img in data:
#     resize = tf.image.resize(img, (256,256))
#     yhat = model.predict(np.expand_dims(resize/255, 0))
#     classes.append(class_names[np.argmax(yhat)])

In [12]:
set(classes)

{'pattern_dependent_exst_sub', 'pattern_direct'}